# Scraping images from mapbox satelitar view  
Mapbox Free tier allows 50,000 requests per month : consider logging how many images have been scraped already to not exceed this limit.  


Pipeline : 
- Define area of interest (bounding box) from coordinates (lat, lon)
- Get the satelitar view of the bounding box from mapbox API
- Crop the images to remove watermark
- Save the images to disk
- Get the bounding boxes of buildings from OSM for the area of interest
- Save the bounding boxes to disk
- Clean overlapping bounding boxes (remove smaller ones or merge them)
- Save the cleaned bounding boxes to disk

The areas of interest have been picked manually :
- Top 20 most populous cities in France
- 100 rural areas coordinates picked randomly across France
- Maybe some random forest to get images without buildings (negative samples)

For the cities : 
- We define a bounding box around the city center with a fixed size (e.g., 10km x 10km)
- We then take random points from this bounding box to scrape images

Then, we can use the pipeline with these points to scrape images and get bounding boxes of buildings.

importing packages

In [ ]:
import os
from pathlib import Path
from dotenv import load_dotenv

CONFIG

In [ ]:
# Load environment variables from .env file
load_dotenv()

# MAPBOX
MAPBOX_ACCESS_TOKEN = os.getenv("MAPBOX_ACCESS_TOKEN")

# Image parameters
WIDTH = 512
HEIGHT = 512 + 30  # extra space to then delete bottom part with watermark
ZOOM = 18  # 18-19 to see buildings, swimming pools clearly

PATH = Path().cwd().parent
BASE_DIR = PATH / "data"
RAW_DIR = BASE_DIR / "raw_images"
CROPPED_DIR = BASE_DIR / "cropped_images"

RAW_POLYGON_DIR = BASE_DIR / "raw_polygons"
CLEANED_POLYGON_DIR = BASE_DIR / "cleaned_polygons"


First, we get all the coordinates of the areas of interest from the two csv files.

In [ ]:
# to import csv files
import pandas as pd

Rural areas coordinates file : "rural_coords.csv"

In [ ]:
FILE_NAME: str = "rural_coords.csv"
rural_coords: pd.DataFrame = pd.read_csv(
    PATH / FILE_NAME,
    sep = ",",
    header=None
)
rural_coords.columns = ["latitude", "longitude"]

In [ ]:
# checking the rural coordinates file
print(f"Number of rural areas: {len(rural_coords)}")
rural_coords.head()

Cities coordinates file : "city_coords.csv"

In [ ]:
FILE_NAME: str = "city_coords.csv"
city_coords: pd.DataFrame = pd.read_csv(
    PATH / FILE_NAME,
    sep = ","
)

In [ ]:
# checking the city coordinates file
print(f"Number of cities: {len(city_coords)}")
city_coords.head()

For the cities, we now create a bounding box around the city center with a fixed size (e.g., 10km x 10km) and then take random points from this bounding box to scrape images.

In [ ]:
import sys
sys.path.append(str(PATH))

from miscellaneous import create_bbox_from_city_coordinates

city_coords_bbox: dict[str, dict[str, float]] = {}
# {city :  {"west": .., "south": .., "east": .., "north": ..}

for _, row in city_coords.iterrows():
    city_name: str = row["city"]
    latitude: float = row["lat"]
    longitude: float = row["lng"]
    bbox: dict[str, float] = create_bbox_from_city_coordinates(
        latitude,
        longitude,
        width=10,
        height=10
    )
    # width and height are in km
    city_coords_bbox[city_name] = bbox


In [ ]:
city_coords_df = pd.DataFrame.from_dict(city_coords_bbox, orient="index")
city_coords_df.head()

Now that we have the bounding boxes for each city, we can get random points from these bounding boxes to scrape images.

In [ ]:
from miscellaneous import get_random_points_in_bbox

city_random_points: dict[str, list[dict[str, float]]] = {}
for city, bbox in city_coords_bbox.items():
    random_points: list[dict[str, float]] = get_random_points_in_bbox(
        bbox,
        number_of_points=10
    )
    # 10 coordinates/images for each city -> 20 * 10 = 200 images
    city_random_points[city] = random_points

In [ ]:
city_random_points_df = pd.DataFrame.from_dict(city_random_points, orient="index")
city_random_points_df.head()